# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. 

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [66]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 1 and isinstance(args[0], list):
            self.rows = len(args[0])
            self.cols = len(args[0][0])
            for row in args[0]:
                if len(row) != self.cols:
                    raise ValueError("All rows must have the same number of columns")
            self.data = args[0]
        elif len(args) == 2:
            self.rows = args[0]
            self.cols = args[1]
            self.data = [[0] * self.cols for _ in range(self.rows)]
        else:
            raise ValueError("Invalid arguments")

    def __getitem__(self, key):
        if isinstance(key, tuple):
            i, j = key
            return self.data[i][j]
        else:
            return self.data[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            i, j = key
            self.data[i][j] = value
        else:
            self.data[key] = value

    def __eq__(self, other):
        if isinstance(other, Matrix):
            return self.data == other.data
        elif isinstance(other, list):
            return self.data == other
        else:
            return False

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.data])
        

In [67]:
# Test cases
# Initializing with size
M1 = Matrix(3, 3)
print("Matrix M1:")
print(M1)

# Initializing with list of lists
M2 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print("\nMatrix M2:")
print(M2)

# Accessing elements
print("\nElement at (1, 1) of M2:", M2[1, 1])

# Setting elements
M2[1, 1] = 0
print("\nMatrix M2 after setting (1, 1) to 0:")
print(M2)

# Equality check
print("\nAre M1 and M2 equal?", M1 == M2)

# Assigning one matrix to another
M1 = M2
print("\nMatrix M1 after assignment:")
print(M1)

# Attempting to assign matrix of different size
try:
    M1 = Matrix([[1, 2], [3, 4]])
except ValueError as e:
    print("\nError:", e)


Matrix M1:
0 0 0
0 0 0
0 0 0

Matrix M2:
1 2 3
4 5 6
7 8 9

Element at (1, 1) of M2: 5

Matrix M2 after setting (1, 1) to 0:
1 2 3
4 0 6
7 8 9

Are M1 and M2 equal? False

Matrix M1 after assignment:
1 2 3
4 0 6
7 8 9


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [68]:
class Matrix:
    def __init__(self, values):
        num_cols = len(values[0])
        for row in values:
            if len(row) != num_cols:
                raise ValueError("All rows must have the same number of columns")
        self.values = values

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.values])

    def shape(self):
        return len(self.values), len(self.values[0])

    def transpose(self):
        transposed = [[self.values[j][i] for j in range(len(self.values))] for i in range(len(self.values[0]))]
        return Matrix(transposed)

    def row(self, n):
        return Matrix([self.values[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.values])

    def to_list(self):
        return self.values

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[n_0:n_1] for row in self.values[m_0:m_1]])

    def __getitem__(self, key):
        if isinstance(key, tuple):
            if isinstance(key[0], slice):
                rows = key[0]
            else:
                rows = slice(key[0], key[0]+1)
            if isinstance(key[1], slice):
                cols = key[1]
            else:
                cols = slice(key[1], key[1]+1)
            return Matrix([row[cols] for row in self.values[rows]])
        else:
            return self.values[key]


In [69]:

# Test cases
matrix = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

print("Matrix:")
print(matrix)
print("Shape:", matrix.shape())
print("Transpose:")
print(matrix.transpose())
print("Row 1:", matrix.row(1))
print("Column 2:",matrix.column(2))
print("Matrix as list:", matrix.to_list())
print("Block (0:2, 1:3):")
print(matrix.block(0, 2, 1, 3))
print("Slice [0:2, 1:3]:")
print(matrix[0:2, 1:3])


Matrix:
1 2 3
4 5 6
7 8 9
Shape: (3, 3)
Transpose:
1 4 7
2 5 8
3 6 9
Row 1: 4 5 6
Column 2: 3
6
9
Matrix as list: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
Block (0:2, 1:3):
4 5
7 8
Slice [0:2, 1:3]:
2 3
5 6


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [70]:
# Standalone functions remain unchanged

def constant(n, m, c):
    return Matrix([[c] * m for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0)

def ones(n, m):
    return constant(n, m, 1)

def eye(n):
    identity_matrix = zeros(n, n)
    for i in range(n):
        identity_matrix[i, i] = 1
    return identity_matrix

In [71]:
# Test cases
print("Constant matrix:")
print(constant(3, 2, 5))

print("\nZeros matrix:")
print(zeros(2, 3))

print("\nOnes matrix:")
print(ones(3, 3))

print("\nIdentity matrix:")
print(eye(4))

Constant matrix:
5 5
5 5
5 5

Zeros matrix:
0 0 0
0 0 0

Ones matrix:
1 1 1
1 1 1
1 1 1


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [57]:
class Matrix:
    def __init__(self, values):
        num_cols = len(values[0])
        for row in values:
            if len(row) != num_cols:
                raise ValueError("All rows must have the same number of columns")
        self.values = values

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.values])

    def shape(self):
        return len(self.values), len(self.values[0])

    def transpose(self):
        transposed = [[self.values[j][i] for j in range(len(self.values))] for i in range(len(self.values[0]))]
        return Matrix(transposed)

    def row(self, n):
        return Matrix([self.values[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.values])

    def to_list(self):
        return self.values

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[n_0:n_1] for row in self.values[m_0:m_1]])

    def __getitem__(self, key):
        if isinstance(key, tuple):
            if isinstance(key[0], slice):
                rows = key[0]
            else:
                rows = slice(key[0], key[0]+1)
            if isinstance(key[1], slice):
                cols = key[1]
            else:
                cols = slice(key[1], key[1]+1)
            return Matrix([row[cols] for row in self.values[rows]])
        else:
            return self.values[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            if isinstance(key[0], slice):
                rows = key[0]
            else:
                rows = slice(key[0], key[0]+1)
            if isinstance(key[1], slice):
                cols = key[1]
            else:
                cols = slice(key[1], key[1]+1)
            if isinstance(value, Matrix):
                value = value.values
            elif not isinstance(value, list):
                value = [[value]]
            self.values[rows] = [self.values[rows][i][:cols] + value[i] + self.values[rows][i][cols+len(value[i]):] for i in range(len(rows))]
        else:
            self.values[key] = value

    def scalarmul(self, c):
        return Matrix([[c * val for val in row] for row in self.values])

    def add(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions are not compatible for addition")
        return Matrix([[self.values[i][j] + other.values[i][j] for j in range(len(self.values[0]))] for i in range(len(self.values))])

    def sub(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions are not compatible for subtraction")
        return Matrix([[self.values[i][j] - other.values[i][j] for j in range(len(self.values[0]))] for i in range(len(self.values))])

    def mat_mult(self, other):
        if self.shape()[1] != other.shape()[0]:
            raise ValueError("Matrix dimensions are not compatible for matrix multiplication")
        result = [[sum(self.values[i][k] * other.values[k][j] for k in range(self.shape()[1])) for j in range(other.shape()[1])] for i in range(self.shape()[0])]
        return Matrix(result)

    def element_mult(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions are not compatible for element-wise multiplication")
        return Matrix([[self.values[i][j] * other.values[i][j] for j in range(len(self.values[0]))] for i in range(len(self.values))])

    def equals(self, other):
        return self.values == other.values

# Test cases
M = Matrix([[1, 2], [3, 4]])
N = Matrix([[5, 6], [7, 8]])

print("Matrix M:")
print(M)

print("\nMatrix N:")
print(N)

print("\nScalar multiplication of M with c=2:")
print(M.scalarmul(2))

print("\nAddition of M and N:")
print(M.add(N))

print("\nSubtraction of N from M:")
print(M.sub(N))

print("\nMatrix multiplication of M and N:")
print(M.mat_mult(N))

print("\nElement-wise multiplication of M and N:")
print(M.element_mult(N))

print("\nAre M and N equal?")
print(M.equals(N))


Matrix M:
1 2
3 4

Matrix N:
5 6
7 8

Scalar multiplication of M with c=2:
2 4
6 8

Addition of M and N:
6 8
10 12

Subtraction of N from M:
-4 -4
-4 -4

Matrix multiplication of M and N:
19 22
43 50

Element-wise multiplication of M and N:
5 12
21 32

Are M and N equal?
False


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [72]:
class Matrix:
    def __init__(self, values):
        num_cols = len(values[0])
        for row in values:
            if len(row) != num_cols:
                raise ValueError("All rows must have the same number of columns")
        self.values = values

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.values])

    def shape(self):
        return len(self.values), len(self.values[0])

    def transpose(self):
        transposed = [[self.values[j][i] for j in range(len(self.values))] for i in range(len(self.values[0]))]
        return Matrix(transposed)

    def row(self, n):
        return Matrix([self.values[n]])

    def column(self, n):
        return Matrix([[row[n]] for row in self.values])

    def to_list(self):
        return self.values

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[n_0:n_1] for row in self.values[m_0:m_1]])

    def __getitem__(self, key):
        if isinstance(key, tuple):
            if isinstance(key[0], slice):
                rows = key[0]
            else:
                rows = slice(key[0], key[0]+1)
            if isinstance(key[1], slice):
                cols = key[1]
            else:
                cols = slice(key[1], key[1]+1)
            return Matrix([row[cols] for row in self.values[rows]])
        else:
            return self.values[key]

    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            if isinstance(key[0], slice):
                rows = key[0]
            else:
                rows = slice(key[0], key[0]+1)
            if isinstance(key[1], slice):
                cols = key[1]
            else:
                cols = slice(key[1], key[1]+1)
            if isinstance(value, Matrix):
                value = value.values
            elif not isinstance(value, list):
                value = [[value]]
            self.values[rows] = [self.values[rows][i][:cols] + value[i] + self.values[rows][i][cols+len(value[i]):] for i in range(len(rows))]
        else:
            self.values[key] = value

    def scalarmul(self, c):
        return Matrix([[c * val for val in row] for row in self.values])

    def add(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions are not compatible for addition")
        return Matrix([[self.values[i][j] + other.values[i][j] for j in range(len(self.values[0]))] for i in range(len(self.values))])

    def sub(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions are not compatible for subtraction")
        return Matrix([[self.values[i][j] - other.values[i][j] for j in range(len(self.values[0]))] for i in range(len(self.values))])

    def mat_mult(self, other):
        if self.shape()[1] != other.shape()[0]:
            raise ValueError("Matrix dimensions are not compatible for matrix multiplication")
        result = [[sum(self.values[i][k] * other.values[k][j] for k in range(self.shape()[1])) for j in range(other.shape()[1])] for i in range(self.shape()[0])]
        return Matrix(result)

    def element_mult(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions are not compatible for element-wise multiplication")
        return Matrix([[self.values[i][j] * other.values[i][j] for j in range(len(self.values[0]))] for i in range(len(self.values))])

    def equals(self, other):
        return self.values == other.values

    # Overloaded operators
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        elif isinstance(other, Matrix):
            return self.mat_mult(other)
        else:
            raise TypeError("Unsupported operand type(s) for *: 'Matrix' and '{}'".format(type(other).__name__))

    def __rmul__(self, other):
        return self.scalarmul(other)

    def __add__(self, other):
        if isinstance(other, Matrix):
            return self.add(other)
        else:
            raise TypeError("Unsupported operand type(s) for +: 'Matrix' and '{}'".format(type(other).__name__))

    def __sub__(self, other):
        if isinstance(other, Matrix):
            return self.sub(other)
        else:
            raise TypeError("Unsupported operand type(s) for -: 'Matrix' and '{}'".format(type(other).__name__))

    def __eq__(self, other):
        if isinstance(other, Matrix):
            return self.equals(other)
        else:
            return False


In [76]:
# Test cases
M = Matrix([[1, 2], [3, 4]])
N = Matrix([[5, 6], [7, 8]])

print("Matrix M:")
print(M)

print("\nMatrix N:")
print(N)

print("\nScalar multiplication of M with c=2(2*M):")
print(2 * M)

print("\nM*2:")
print(M * 2)

print("\nMatrix multiplication of M and N:")
print(M * N)

print("\nAddition of M and N:")
print(M + N)

print("\nSubtraction of N from M:")
print(M - N)

print("\nEquality check: M == N")
print(M == N)

print("\nAssigning M to N:")
N=M
print(N)

Matrix M:
1 2
3 4

Matrix N:
5 6
7 8

Scalar multiplication of M with c=2(2*M):
2 4
6 8

M*2:
2 4
6 8

Matrix multiplication of M and N:
19 22
43 50

Addition of M and N:
6 8
10 12

Subtraction of N from M:
-4 -4
-4 -4

Equality check: M == N
False

Assigning M to N:
1 2
3 4


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [79]:
# Define the matrices A, B, and C
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[9, 10], [11, 12]])

# Identity matrix
I = Matrix(values=[[1, 0], [0, 1]])

# Calculate (AB)C and A(BC)
AB_C = (A * B) * C
A_BC = A * (B * C)

# Calculate A(B + C) and AB + AC
B_plus_C = B + C
A_B_plus_C = A * B_plus_C
A_B_plus_AC = (A * B) + (A * C)

# Check if AB is not equal to BA
AB_not_equal_BA = A * B != B * A

# Calculate AI and A
AI = A * I
A_equals_A = A == A

In [80]:
# Print the results
print("(AB)C equals A(BC):", AB_C == A_BC)
print("A(B + C) equals AB + AC:", A_B_plus_C == A_B_plus_AC)
print("AB is not equal to BA:", AB_not_equal_BA)
print("AI equals A:", A_equals_A)

(AB)C equals A(BC): True
A(B + C) equals AB + AC: True
AB is not equal to BA: True
AI equals A: True
